In [1]:
import re
import pandas as pd

In [5]:
# 문장 전처리를 위한 함수
def text_preprocessor(text) :
    # 문자열내의 html 태그를 제거한다.
    text = re.sub('<[^>*>]', '', text)
    # 문자열에서 이모티콘을 찾아낸다.
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)|\^.?\^', text)
    # 문장에서 특수문자를 제거하고
    # 문자열을 소문자로 변경하고
    # 추출한 이모티콘을 붙혀준다.
    text = re.sub('[\W]+', ' ', text.lower() + ' '.join(emoticons).replace('-', ''))
    return text         


In [8]:
# 전처리
df = pd.read_csv('movie_review.csv')
df['review']= df['review'].apply(text_preprocessor)

df.to_csv('refind_movie_review.csv', index=False)

### 학습

In [10]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [12]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk

In [13]:
# stopword 단어 사전을 다운로드 받는다.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
# stopword 데이터를 가져온다.
stop = stopwords.words('english')
# stop

In [17]:
# 단어 형태소 분석을 위한 객체 생성
porter = PorterStemmer()

In [18]:
# 형태소를 분석하는 함수
def tokenizer_porter(text) :
    # 띄어쓰기를 기준으로 분리한다.
    word_list = text.split()
    # 형태소 분석 처리
    word_list2 = [porter.stem(word) for word in word_list]
    return word_list2

In [20]:
# text = 'runners like running and thus they run'
# a1 = tokenizer_porter(text)
# a1

### 학습

In [21]:
# 데이터를 준비한다.
df = pd.read_csv('refind_movie_review.csv')

In [25]:
# 학습데이터와 검증데이터로 나눈다.
X_train = df.loc[:35000 - 1, 'review'].values
y_train = df.loc[:35000 - 1, 'sentiment'].values

X_test = df.loc[35000:, 'review'].values
y_test = df.loc[35000:, 'sentiment'].values

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle

In [27]:
# 단어사전을 생성하는 모델
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_porter, stop_words=stop)
# 학습 모델
model1 = LogisticRegression()

# 파이프라인 연결
pipe = Pipeline([('vect', tfidf), ('clf', model1)])

In [29]:
pipe.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...\b',
                                 tokenizer=<function tokenizer_porter at 0x000001F3E70C30D8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_

In [30]:
# 테스트
y_pred = pipe.predict(X_test)
print(f'정확도 : {accuracy_score(y_test, y_pred)}')

정확도 : 0.8961333333333333


In [31]:
# 학습이 완료된 모델을 저장한다.
with open('pipe.dat', 'wb') as fp :
    pickle.dump(pipe, fp)